# Set up

In [1]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip ModelNet10.zip

Streaming output truncated to the last 5000 lines.
  inflating: ModelNet10/monitor/test/monitor_0558.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0558.off  
  inflating: ModelNet10/monitor/test/monitor_0559.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0559.off  
  inflating: ModelNet10/monitor/test/monitor_0560.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0560.off  
  inflating: ModelNet10/monitor/test/monitor_0561.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0561.off  
  inflating: ModelNet10/monitor/test/monitor_0562.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0562.off  
  inflating: ModelNet10/monitor/test/monitor_0563.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0563.off  
  inflating: ModelNet10/monitor/test/monitor_0564.off  
  inflating: __MACOSX/ModelNet10/monitor/test/._monitor_0564.off  
  inflating: ModelNet10/monitor/test/monitor_0565.off  
  inflating: __MACOSX/ModelNet10

In [2]:
!pip install trimesh
!pip install pyvista

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.6/676.6 KB 27.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 MB 12.1 MB/s eta 0:00:00


# Load Dataset

In [6]:
import os
import glob
import trimesh
import numpy as np

DATA_DIR = "ModelNet10/*"

def parse_dataset(num_points=4096):
  print("Loading dataset")
  
  objects = []

  folders = glob.glob(DATA_DIR)

  for i, folder in enumerate(folders):
      train_points = []
    
      train_files = glob.glob(folder + "/train/*")

      for f in train_files:
        train_points.append(trimesh.load(f).sample(num_points))

      objects.append(train_points)

  return objects

# GAN

In [ ]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Input, BatchNormalization, Activation, LeakyReLU, Flatten, Dropout
from keras.optimizers import Adam
import pyvista as pv

data = parse_dataset()
epochs = 10000

def discriminator():
    input = Input(shape=(4096,3)) 

    x = Flatten()(input)
    x = Dropout(0.4)(x)

    x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.4)(x)

    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.4)(x)

    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)

    output = Dense(1, activation="sigmoid")(x)

    model = Model(input, output)

    return model

def generator():
    model = Sequential()

    model.add(Dense(512, input_shape=(100,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())

    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())

    model.add(Dense(4096 * 3, activation='tanh'))  # remove UpSampling1D

    model.add(Reshape((4096, 3)))  # change to output shape to (None, 4096, 3)

    return model

def gan(discr, gener):
    discr.trainable = False

    model = Sequential([
        gener,
        discr
    ])

    model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss="binary_crossentropy")

    return model

def train():
    discr = discriminator()
    gener = generator()

    discr.compile(optimizer='adam', loss='binary_crossentropy')
    
    gan_model = gan(discr, gener)

    for epoch in range(epochs):
        real = data[0][0]
        real = np.reshape(real, (1, real.shape[0], real.shape[1]))

        noise = np.random.normal(0, 1, size=(1, 100))
        fake = gener.predict(noise)

        d_loss_real = discr.train_on_batch(real, np.array([1]))
        d_loss_fake = discr.train_on_batch(fake, np.array([1]))

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (4096, 3))
        g_loss = gan_model.train_on_batch(noise, np.ones((1,4096, 3)))

        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

train()

Loading dataset
